# Scraping du site beerwulf pour enrichir la base de données

- Nous allons scrapper les données relatives aux bières sur le site www.beerwulf.com. L'avantage de ce site est qu'il dispose de données précises sur les bières : des données physico chimiques, mais aussi des indicateurs subjectifs exhaustifs (couleur de la bière, son goût, ou encore sa longueur). Nous utilisons le site https://www.beerwulf.com/fr-be/c/bieres?page=1&container=Bouteille,Canette pour afficher toutes les bières ainsi que les canettes proposées.
- Avant toute chose, il est nécessaire d'explorer la page https://www.beerwulf.com/robots.txt afin de s'assurer que le web-scrapping des données des bières est autorisé. **Ici, il se trouve que c'est le cas.**

On peut alors commencer à importer les librairies et les packages nécessaires au scraping

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np 

options = Options()
options.headless = True # On "n'affiche pas" la recherche internet
options.add_argument("--window-size=1920,1200")
DRIVER_PATH = '/Applications/chromedriver'

### Étape 1 : obtenir de l'url de toutes les bières du site de www.berewulf.com
Le catalogue comporte 25 pages de bières. On peut donc automatiser la collecte des urls grâce au package `selenium` puisque les pages sont codées en Javascript. Bs4 ne peut pas aller scraper directement les données recherchées. Par la suite, nous utilisons le moteur de recherche de Google Chrome.

In [2]:
# Boucle qui parcourt toutes les pages du site et va collecter le lien `href` de toutes les bières.
nb_pages = 25
liste_urls = []

for i in range(nb_pages):
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get('https://www.beerwulf.com/fr-be/c?page='+str(i+1)+'&container=Bouteille,Canette&segment=Toutes%20les%20bi%C3%A8res&routeQuery=c')
    elems = driver.find_elements_by_xpath("//div[@id='product-items-container']/a")
    for elem in elems:
        liste_urls.append(elem.get_attribute('href'))

#Vérification
print('Il y a '+str(len(liste_urls))+' urls collectés.')

Il y a 1164 urls collectés.


### Étape 2 : obtenir les caractéristiques de toutes les bières du site, et les stocker dans un dictionnaire
Nous disposons donc de l'url de toutes les bières, donc nous pouvons nous rendre sur ces pages internet grâce à `selenium`. Nous pouvons à présent scraper leurs caractéristiques individuelles.

Pour ce faire, nous avons d'abord besoin de définir des fonctions qui permettront d'aller scraper les caractéristiques des bières. Les données ne sont pas déjà bien rangées dans un tableau, il faut aller les scraper individuellement grâce à leur **"Xpath"**, c'est-à-dire le chemin qui permettra à `selenium` d'accéder à la donnée si ce n'est pas possible avec `BeautifulSoup`.

In [3]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

def valeur_text(xpath):
    """
    Cette fonction scrape des caractéristiques de la bière (degré d'alccolémie, nom de 
    la brasserie, etc) au format text si cette dernière est présente sur la page. 
    """
    try : 
        return driver.find_element_by_xpath(xpath).text
    except (NoSuchElementException, StaleElementReferenceException) as e:
        return float('nan')

def valeur_percent(xpath):
    """
    Cette fonction scrape des caractéristiques de la bière exprimées en pourcentage
    (acidité, amertume, etc) si cette dernière est présente sur la page. 
    """
    try : 
        return driver.find_element_by_xpath(xpath).get_attribute('data-percent')
    except (NoSuchElementException, StaleElementReferenceException) as e:
        return float('nan')


# Scraper l'IBU d'une bière ne peut être entièrement réalisé avec selenium, il faut donc utiliser bs4
def get_ibu(url):
    """
    url : url de la bière selectionnee
    Fonction renvoie l'IBU de la bière grâce à la librairie BeautifulSoup
    """
    request = urllib.request.Request(url, headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    html = urllib.request.urlopen(request)
    html_soup = BeautifulSoup(html, 'html.parser')
    try:
        return html_soup.find_all('strong')[0].text
    except IndexError:
        return float('nan')


On peut désormais parcourir les pages des différentes bières de www.beerwulf.com et stocker leurs caractéristiques dans un dictionnaire.

In [4]:
dico_all_beers = {}

for i in range(len(liste_urls)):
    driver.get(liste_urls[i])

    beer_name = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[1]/h1")
    beer_style = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[1]/a")
    country = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[4]")
    brewery_name = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[5]/a")
    bottle = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[2]")
    abv = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[3]")
    ibu = get_ibu(liste_urls[i])
    intensite = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[1]/dd/div")
    beer_type = valeur_percent("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[1]/a")
    longueur = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[3]/dd/div")
    acidite = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[4]/dd/div")
    amertume = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[5]/dd/div")
    price = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[3]/div[2]/div/span")
    note = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[3]/div[1]/span")

    liste_caractéristiques = [beer_name, beer_style, country, brewery_name, bottle, abv, ibu, intensite, longueur, acidite, amertume, price, note]
        
    dico_all_beers[beer_name] = liste_caractéristiques

# Affichage des 5 premières bières scrapées
print(list(dico_all_beers.items())[:5])
print('\nNous avons scrapé '+str(len(dico_all_beers))+' bières.')

[('Walhalla Aphrodite Raspberry Berliner Weisse', ['Walhalla Aphrodite Raspberry Berliner Weisse', 'Bière Sour', 'Pays-Bas', 'Walhalla Craft Beer', '33 cl', '4,0%', '-', '0', '40', '40', '0', nan, nan]), ('Affligem Blond 0.0%', ['Affligem Blond 0.0%', 'Bière Blonde', 'Belgique', 'Affligem', '30 cl', '0,0%', nan, nan, nan, nan, nan, '€ 1,59', '(2,84)']), ('Affligem Blond', ['Affligem Blond', 'Bière Blonde', 'Belgique', 'Affligem', '30 cl', '6,8%', '-', '60', '60', '0', '40', nan, '(3,76)']), ('Affligem Tripel', ['Affligem Tripel', 'Bière Triple', 'Belgique', 'Affligem', '30 cl', '9,0%', '-', '60', '60', '20', '60', '€ 1,79', '(3,57)']), ('Ardwen Woinic Rouge', ['Ardwen Woinic Rouge', 'Bière Fruitée', 'France', 'Ardwen', '33 cl', '8,0%', '-', '60', '100', '40', '20', '€ 2,99', '(3,07)'])]

Nous avons scrapé 1145 bières.


**Bilan** : 1164 urls scrapés ont permis de scraper 1145 bières ensuite. Il n'y a pas autant de bière que d'urls en raison de mise à jour du site www.beerwulf.com : certaines apparaissent en promotion et cela change la configuration de la page, par exemple. 

### Étape 3 : on convertit ensuite notre dictionnaire en dataframe

In [59]:
df_beers = pd.DataFrame.from_dict(dico_all_beers, orient='index') #faire une table à partir d'un dictionnaire
df_beers.columns = ['beer_name', 'beer_style', 'country', 'brewery_name', 'bottle', 'abv', 'ibu', 'intensite', 'longueur', 'acidite', 'amertume', 'price', 'note']
df_beers = df_beers.reset_index(drop=True)
df_beers.head(5)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33 cl,"4,0%",-,0,40,40,0,NaN,NaN
1,Affligem Blond 0.0%,Bière Blonde,Belgique,Affligem,30 cl,"0,0%",NaN,NaN,NaN,NaN,NaN,"€ 1,59","(2,84)"
2,Affligem Blond,Bière Blonde,Belgique,Affligem,30 cl,"6,8%",-,60,60,0,40,NaN,"(3,76)"
3,Affligem Tripel,Bière Triple,Belgique,Affligem,30 cl,"9,0%",-,60,60,20,60,"€ 1,79","(3,57)"
4,Ardwen Woinic Rouge,Bière Fruitée,France,Ardwen,33 cl,"8,0%",-,60,100,40,20,"€ 2,99","(3,07)"


### Étape 4 : on nettoie les données pour leur exploitation future 

On convertir les chaines de caractères en `float` pour plus tard traiter les données

In [60]:
# Fonction qui nettoie les données scrapées au format text. Cela permet ensuite de convertir les données chiffrées en float 
def clean(text):
    text = text.replace(' cl','').replace('%','').replace('(','').replace(')','').replace('€ ','').replace(',','.')
    return text

# On nettoie les entrées du dataframe
for i in range(len(df_beers)):
    for j in ['bottle', 'abv', 'intensite', 'longueur', 'acidite', 'amertume', 'ibu', 'price', 'note']:
        if df_beers[j][i] == '-': # Seules les données relatives à 'ibu' ont des valeurs '-'
            df_beers[j][i] = np.nan
        elif type(df_beers[j][i]) == str and len(df_beers[j][i]) > 6: # Nettoyage des données non chiffrées
            df_beers[j][i] = np.nan
        elif type(df_beers[j][i]) !=  float: 
            df_beers[j][i] = float(clean(df_beers[j][i])) # Nettoyage pour ensuite convertir au format float

df_beers.head(5)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33,4,NaN,0,40,40,0,NaN,NaN
1,Affligem Blond 0.0%,Bière Blonde,Belgique,Affligem,30,0,NaN,NaN,NaN,NaN,NaN,1.59,2.84
2,Affligem Blond,Bière Blonde,Belgique,Affligem,30,6.8,NaN,60,60,0,40,NaN,3.76
3,Affligem Tripel,Bière Triple,Belgique,Affligem,30,9,NaN,60,60,20,60,1.79,3.57
4,Ardwen Woinic Rouge,Bière Fruitée,France,Ardwen,33,8,NaN,60,100,40,20,2.99,3.07


In [62]:
df_beers.describe()

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note
count,1144,1136,1143,771,1144.0,1143.0,649.0,1078.0,1078.0,1078.0,1078.0,970.00,1061.00
unique,1144,19,27,164,12.0,86.0,79.0,6.0,6.0,6.0,6.0,65.00,149.00
top,St-Feuillien Quadrupel,IPA,Pays-Bas,Brasserie Oedipus,33.0,6.5,30.0,60.0,60.0,0.0,40.0,2.89,3.49
freq,1,181,381,19,943.0,87.0,52.0,641.0,587.0,514.0,388.0,92.00,28.00


## 1ère analyse
Ces statistiques descriptives nous permettent de visualiser plusieurs problèmes : 
- La valeur minimum de `abv` est 0, ce qui veut dire que nous avons scrapé des bières sans alcool. Néanmoins, la valeur maximale (16.5°) semble cohérente.
- La valeur maximale de l'`ibu` est louche.
- L'`acidite` un pourcentage dont la valeur maximale est de 80%. Cette varaible n'a pas l'amplitude de `intensite`, `longueur` et `amertume`.
- Le `prix` semble honnête, mais la valeur maximale semble étrange. De plus, le prix n'est pas uniformisé dans le sens où il correspond à un volume (`bottle`) précis. Il faudra donc ramener le tout au prix de la pinte de bière. 
- La `note` de satisfation de la bière est bien comprise entre 0 et 5 mais son écart-type est très faible. On aura donc du mal à distinguer les très bonnes bières et les bonnes bières.

### On nettoie le dataframe afin de ne garder que les bières sans alcool

In [63]:
bieres_sans_alcool = df_beers[df_beers['abv'].isin([0,np.nan])] 
bieres_sans_alcool.head(8)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note
1,Affligem Blond 0.0%,Bière Blonde,Belgique,Affligem,30,0,NaN,NaN,NaN,NaN,NaN,1.59,2.84
51,Heineken 0.0,Lager,Pays-Bas,Heineken Brewery,30,0,16,40,40,40,40,1.49,2.3
162,"Budels Malty Dark 0,0%",Bière Bock,Pays-Bas,Budelse Brouwerij,30,0,NaN,40,60,20,20,1.79,1.85
180,Brand Weizen 0.0,Bière Weiss,Pays-Bas,Brand,30,0,12,60,60,20,20,1.79,2.96
183,Palm 0.0,Pale Ale,Belgique,Brouwerij Palm,25,0,NaN,60,60,20,40,1.89,2.06
217,Braxzz Porter,Bière Porter & Stout,Pays-Bas,NaN,33,0,50,60,80,0,40,2.49,2.51
352,Brand IPA 0.0%,IPA,Pays-Bas,Brand,30,0,NaN,60,0,0,0,1.89,2.96
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


D'ailleurs, on remarque que les bières sans alcool n'ont pas de "bonne" note. On les supprime du dataframe.

In [64]:
df_beers = df_beers.drop([df_beers[df_beers['abv'].isin([0,np.nan])].index[i] for i in range(len(df_beers[df_beers['abv'].isin([0,np.nan])]))])
df_beers = df_beers.reset_index(drop = True)

In [65]:
df_beers.isnull().sum()

beer_name         0
beer_style        7
country           0
brewery_name    368
bottle            0
abv               0
ibu             485
intensite        62
longueur         62
acidite          62
amertume         62
price           170
note             79
dtype: int64

On peut à présent ajouter une colonne égale au prix de la pinte de cette bière, d'après le site www.beerwulf.com. 

Ce sera utile pour comparer les prix par la suite.

In [66]:
df_beers['prix_de_la_pinte'] = 0.0

for i in range(len(df_beers)):
    if df_beers['price'][i] != np.nan:
        df_beers['prix_de_la_pinte'][i] = round(50*df_beers['price'][i]/df_beers['bottle'][i],2)
    else:
        df_beers['prix_de_la_pinte'][i] = np.nan

df_beers.head(3)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33,4,NaN,0,40,40,0,NaN,NaN,NaN
1,Affligem Blond,Bière Blonde,Belgique,Affligem,30,6.8,NaN,60,60,0,40,NaN,3.76,NaN
2,Affligem Tripel,Bière Triple,Belgique,Affligem,30,9,NaN,60,60,20,60,1.79,3.57,2.98


### Formatage des données : convertir les données relatives à `intensite`, `longueur`, `acidite`, `amertume` en une note sur 5 pour pouvoir ensuite faire le lien avec l'autre base de données.
Les notes des bières concernant l'`intensite`, la `longueur` et l'`amertume` vont de 0 à 100%, tandis que la note de l'acidité varie entre 0 et 80%. On adapte donc le formatage.

In [67]:
for i in range(len(df_beers)):
    for j in ['intensite', 'longueur','amertume']:
        if df_beers[j][i] != np.nan:
            df_beers[j][i] = df_beers[j][i]/20
    if df_beers['acidite'][i] != np.nan:
        df_beers['acidite'][i] = df_beers['acidite'][i]/16

df_beers.head(10)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33,4,NaN,0,2,2.5,0,NaN,NaN,NaN
1,Affligem Blond,Bière Blonde,Belgique,Affligem,30,6.8,NaN,3,3,0,2,NaN,3.76,NaN
2,Affligem Tripel,Bière Triple,Belgique,Affligem,30,9,NaN,3,3,1.25,3,1.79,3.57,2.98
3,Ardwen Woinic Rouge,Bière Fruitée,France,Ardwen,33,8,NaN,3,5,2.5,1,2.99,3.07,4.53
4,Atlantic Blanche des Gabariers,Bière Blanche,France,Brasserie des Gabariers,33,5,NaN,2,3,1.25,1,2.79,2.93,4.23
5,Asahi Super Dry,Lager,Japon,NaN,33,5.2,16,3,2,0,2,2.39,3.07,3.62
6,Atlantic Rubis des Gabariers,Bière Ambrée,France,Brasserie des Gabariers,33,6,NaN,2,4,3.75,1,2.99,2.79,4.53
7,Atlantic Dorée des Gabariers,Bière Ambrée,France,Brasserie des Gabariers,33,5.5,NaN,2,3,1.25,2,2.79,2.84,4.23
8,Bacchus Framboos,Bière Fruitée,Belgique,Brouwerij Van Honsebrouck,38,5,NaN,4,3,3.75,1,3.49,3.6,4.59
9,Baladin Nora,Bière Blonde,Italie,Baladin,33,6.8,11,3,3,0,1,3.49,3.63,5.29


# Statistiques descriptives

Exportation du tableau en tant que fichier csv pour ensuite être traité

In [68]:
df_beers.to_csv('data_beerwulf') 